# Autodesk Inventor API
Based on a COM technology with (VB.NET, C++, C# and Python) to develop a plug-in.

## Reference
> C:\Windows\Microsoft.NET\assembly\GAC_MSIL\Autodesk.Inventor.Interop\
> C:\Program Files\Autodesk\Inventor 20xx\Bin\Public Assemblies\

In [ ]:
using System.Windows.Forms;
using Inventor;

Inventor.Application _invApp;
bool _started = false;

## Create and open Inventor instance
Connect to or start an Inventor session 

In [ ]:
try
{
    _invApp = (Inventor.Application)Marshal.GetActiveObject("Inventor.Application");

}
catch (Exception ex)
{
    try
    {
        Type invAppType = Type.GetTypeFromProgID("Inventor.Application");

        _invApp = (Inventor.Application)System.Activator.CreateInstance(invAppType);
        _invApp.Visible = true;

        //Note: if the Inventor session is left running after this //form is closed, there will still an be and Inventor.exe running. 
        //We will use this Boolean to test in Form1.Designer.cs in the dispose method whether or not the Inventor App should be shut down when the form is closed.
        _started = true;

    }
    catch (Exception ex2)
    {
        MessageBox.Show(ex2.ToString());
        MessageBox.Show("Unable to get or start Inventor");
    }
}

In [ ]:
//Check whether Document is open and Verify it is assembly documnent
if (_invApp.Documents.Count == 0) { MessageBox.Show("Need to open an Assembly document"); return; }
if (_invApp.ActiveDocument.DocumentType != DocumentTypeEnum.kAssemblyDocumentObject)
{ MessageBox.Show("Need to have an Assembly document active"); return; }

## Open Part or Assembly Document

In [ ]:
AssemblyDocument asmDoc = default(AssemblyDocument);
PartDocument partDoc = default(PartDocument);
asmDoc = (AssemblyDocument)_invApp.ActiveDocument;

### Check and find the Selected component list

In [ ]:
if (asmDoc.SelectSet.Count == 0) { MessageBox.Show("Need to select a Part or Sub Assembly"); return; }
SelectSet selSet = default(SelectSet);
selSet = asmDoc.SelectSet;

//Loop though Selected Component and Change the visibilty of the component
try
{
    ComponentOccurrence compOcc = default(ComponentOccurrence);
    foreach (object obj in selSet)
    {
        compOcc = (ComponentOccurrence)obj;
        System.Diagnostics.Debug.Print(compOcc.Name);
        compOcc.Visible = false;
    }
}
catch (Exception ex) { MessageBox.Show("Is the selected item a Component? \n" + ex.ToString()); return; }

### Attribute Sets
Attribute Set here is the selected parts or sub-assemblies
#### Add Attribute Sets
> Instead of directly hiding the selected components, you’ll now modify the code to “tag” your components for later hiding or display. Attaching attributes provided by the Inventor API will allow you to identify and perform different operations on your tagged components.

Three Inventor API classes that are used to add an Attribute to entities, one of which is the AttributeSets collection.
> Attribute --> AttritubeSet --> AttributeSets

In [ ]:
AttributeSets attbSets = compOcc.AttributeSets;
// Add the attributes to the ComponentOccurrence
if (!attbSets.NameIsUsed["myPartGroup"])
{
    AttributeSet attbSet = attbSets.Add("myPartGroup");
    Inventor.Attribute attb = attbSet.Add("PartGroup1", ValueTypeEnum.kStringType, "Group1");
}

#### Remove Attribute Sets

In [ ]:
AttributeSets attbSets = compOcc.AttributeSets;
if (attbSets.NameIsUsed["myPartGroup"] ) { attbSets["myPartGroup"].Delete(); }

#### Remove All from Attribute Set
Gets all the components with your AttributeSet attached and deletes the AttributeSet from each one (rather than deleting it only from the selected components). In this Sub, after you had an ObjectCollection with all of the components in the Assembly with your AttributeSet attached, you used a For Next loop to delete the AttributeSet from each component. You also needed to make each component visible: after the Sub had finished its work, these components no longer had your AttributeSet attached, and so would no longer be affected by the “Show group” button.

In [ ]:
AttributeManager attbMan = asmDoc.AttributeManager;
ObjectCollection objsCol = default(ObjectCollection);
objsCol = attbMan.FindObjects("myPartGroup", "PartGroup1", "Group1");

AttributeSets attbSets;
foreach (ComponentOccurrence obj in objsCol)
{
    attbSets = (AttributeSets)obj.AttributeSets;
    attbSets["myPartGroup"].Delete();
    obj.Visible = true;
}


#### Show/Hide Attribute Sets
Attribute Visibility Switch ON/OFF

In [ ]:
try
{
    AttributeManager attbMan = asmDoc.AttributeManager;
    ObjectCollection objsCol = default(ObjectCollection);
    objsCol = attbMan.FindObjects("myPartGroup", "PartGroup1", "Group1");
    ComponentOccurrence compOcc = default(ComponentOccurrence);
    foreach (object obj in objsCol)
    {
        compOcc = (ComponentOccurrence)obj;
        // Toggle the visibility of the Component Occurrence
        compOcc.Visible = !compOcc.Visible; //(Can Be added to Params like compOcc.Visible = hide)
    }
}
catch (Exception ex) { MessageBox.Show("Problem hiding component"); }